# Committor Analysis

(Note: some of this requires OpenPathSampling 0.9.1 or later)

The particular approach used in this notebook is to save each shot as a separate "experiment," where each of the experiements is a tuple matching initial snapshot to final state. That was a peculiarity of the project from which this example originated. Typically, you could just go directly to the analysis of the shooting points. However, if you already have data from elsewhere that can be put into this format, the next notebook shows you how you can build a committor analysis from such a list.

In [1]:
import openpathsampling as paths

In [2]:
%%time
simulation_storage = paths.AnalysisStorage("committor_simulation.nc")
C_7eq = simulation_storage.volumes['C_7eq']
alpha_R = simulation_storage.volumes['alpha_R']

CPU times: user 1min 44s, sys: 29.6 s, total: 2min 14s
Wall time: 48min 2s


In [3]:
analyzer = paths.ShootingPointAnalysis(steps=None, states=[C_7eq, alpha_R])

In [4]:
%%time
# the shooting point snapshot for each
shooting_pts = [analyzer.step_key(step) for step in simulation_storage.steps]

CPU times: user 57.6 s, sys: 1.45 s, total: 59.1 s
Wall time: 2min 29s


In [5]:
%%time
# get the final states from each partial trajectory
final_states_list = [analyzer.analyze_single_step(step) for step in simulation_storage.steps]

CPU times: user 4min 54s, sys: 4.4 s, total: 4min 59s
Wall time: 11min 6s


In [6]:
# check that there's only one state per item in that list
for f in final_states_list:
    assert len(f) == 1
# flatten the list
final_states = [f[0] for f in final_states_list]

In [7]:
experiments = zip(shooting_pts, final_states)

In [8]:
output_storage = paths.Storage("committor_results.nc", "w")

In [9]:
output_storage.save(C_7eq)
output_storage.save(alpha_R)
output_storage.tag['experiments'] = experiments

In [10]:
output_storage.sync()
output_storage.close()